In [1]:

%cd /home/ryn_mote/Misc/vlm_with_pooled_for_text_genrec/

import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

from model import model, tokenizer


/home/ryn_mote/Misc/vlm_with_pooled_for_text_genrec


/home/ryn_mote/anaconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at OpenGVLab/InternVL2-4B were not used when initializing InternVLChatModel: ['vision_model.embeddings.class_embedding', 'vision_model.embeddings.patch_embedding.bias', 'vision_model.embeddings.patch_embedding.weight', 'vision_model.embeddings.position_embedding', 'vision_model.encoder.layers.0.attn.proj.bias', 'vision_model.encoder.layers.0.attn.proj.weight', 'vision_model.encoder.layers.0.attn.qkv.bias', 'vision_model.encoder.layers.0.attn.qkv.weight', 'vision_model.encoder.layers.0.ls1', 'vision_model.encoder.layers.0.ls2', 'vision_model.encoder.layers.0.mlp.fc1.bias', 'vision_model.encoder.layers.0.mlp.fc1.weight', 'vision_model.encoder.layers.0.mlp.fc2.bias', 'vision_model.encoder.layers.0.mlp.fc2.weight', 'vision_model.encoder.layers.0.norm1.bias', 'vision_model.encoder.layers.0.norm1.weight', 'vision_model.encoder.layers.0.norm2.bias', 'vision_model.encoder.layers.0.norm2.weight', 'vision_model.encoder.layers.1.attn.proj.bias', 'vision_model.

Sequential(
  (0): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (1): Linear(in_features=512, out_features=3072, bias=True)
  (2): GELU(approximate='none')
  (3): Linear(in_features=3072, out_features=3072, bias=True)
)
Training: [Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1

In [ ]:

from model import load_image

# set the max number of tiles in `max_num`
with torch.cuda.amp.autocast(True, torch.bfloat16):
    pixel_values = load_image('/home/ryn_mote/Downloads/A polaroid of Silver Forgettings -- This flower does not exist.png', max_num=1).cuda()
    emb1 = model.extract_feature(pixel_values.to(torch.bfloat16))

    pixel_values = load_image('/home/ryn_mote/Downloads/20230707_160218.jpg', max_num=1).cuda()
    emb2 = model.extract_feature(pixel_values.to(torch.bfloat16))

    w = [3, 5]
    embs = (emb1*w[0]+emb2*w[1]) / sum(w)

    generation_config = dict(max_new_tokens=32, do_sample=False,)

    # single-image single-round conversation (单图单轮对话)
    question = '\n '
    response = model.chat(tokenizer, pixel_values, question, generation_config, visual_features=embs.to(torch.bfloat16))
    print(f'User: {question}\nAssistant: {response}')
